In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
# sys.path.append("../../")

sys.path.append("../")
# sys.path.append("..")
from early_exit.patching.method_patching import replace_attention_layers, set_transformer_early_exit_mode
from shared_utils.generate import format_conversation, transform_conversations
from early_exit.util import module_name_is_layer_base
import numpy as np
from early_exit.util import get_model
from shared_utils.load import get_tokenizer, configs_from_yaml
from shared_utils.generate import generate_text
import random
from early_exit_teacher.visualization import visualize_tokens_by_exit_layer, create_html_visualization
from IPython.display import HTML, display
from early_exit.util import module_name_is_layer_base
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")
import torch.nn.functional as F
import pandas as pd
from transformers.models.qwen2.modeling_qwen2 import apply_rotary_pos_emb, repeat_kv, Qwen2Attention

Disabled automatic differentiation


In [3]:
# Model configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

print(f"Loading model: {model_name}")
print(f"Device: {device}")

tokenizer = get_tokenizer(model_name)
model_config_path = "/project/project_465001340/fair_stuff/externalization/config_deepseek.yaml"   # args.model_config_path
config = configs_from_yaml(model_config_path, tokenizer.eos_token_id)

model = get_model(model_name, config['model'], device)
model = replace_attention_layers(model, config['lora'], device)
# set_transformer_early_exit_mode(model, 'off')

# Load tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    

config['generation']['max_new_tokens'] = 10

print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")



prompt = "Explain the concept of recursion in programming."
system_prompt = "You are a helpful programming tutor."
prefiller = ""

pre_transformed_conversation = format_conversation(user_prompts = [prompt], system_prompt=system_prompt)
formatted_prompt = transform_conversations(pre_transformed_conversation, prefiller)[0]
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
input_ids = inputs.input_ids

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
Device: cuda
address this hack!
g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

trainable params: 2,179,072 || all params: 1,779,276,294 || trainable%: 0.1225
Tokenizer loaded. Vocab size: 151643
EOS token: <｜end▁of▁sentence｜> (ID: 151643)
transform_conversations currently only for Deepseek models!


In [12]:

from early_exit.util import module_name_is_layer_base
early_exit_layer_idxs = []
for name, module in model.named_modules():
    if module_name_is_layer_base(name):
        layer_idx = int(name.split('.')[-1])
        early_exit_layer_idxs.append(layer_idx)

early_exit_layer_idxs = torch.tensor(early_exit_layer_idxs, dtype = torch.int32)  # Add inf for final layer

In [32]:
def set_early_exit_layer(step):
    if step > 2 and step < 10: return 25
    return 27

set_transformer_early_exit_mode(model, 'off')
generated_tokens = []
max_new_tokens = 20
generated = inputs["input_ids"]
chosen_exit_layers = []
all_student_logits = []
all_teacher_logits = []
for step in range(max_new_tokens):  # generate 10 tokens
    if step == 0:
        student_outputs = model(generated, use_cache=True)
        student_logits = student_outputs.logits
        student_cache = student_outputs.past_key_values
        teacher_outputs = model(generated, use_cache=True)
        teacher_cache = teacher_outputs.past_key_values
        teacher_logits = teacher_outputs.logits
    else:
        # Pass only the new token and the cache
        student_outputs = model(
            next_token,
            past_key_values=student_cache,
            use_cache=True,
            output_hidden_states=True
        )
        early_exit_layer = set_early_exit_layer(step)
        hidden_states = student_outputs.hidden_states
        hidden_states = torch.stack(student_outputs.hidden_states)[1:]
        # print(f"Hidden states shape = {hidden_states.shape}")
        exit_hidden_state = hidden_states[early_exit_layer]
        student_logits = model.lm_head(exit_hidden_state)
        all_student_logits.append(student_logits)
        student_cache = student_outputs.past_key_values  # updated with new token
        # print(len(past_key_values))
        for layer_idx in range(early_exit_layer + 1, 28):
            layer = model.base_model.model.model.layers[layer_idx]
            normed_hidden = layer.input_layernorm(exit_hidden_state)

            # Project to K and V using this layer's projections
            key_states = layer.self_attn.k_proj(normed_hidden)
            value_states = layer.self_attn.v_proj(normed_hidden)
            
            # Reshape for multi-head attention
            num_key_value_heads = layer.self_attn.config.num_key_value_heads
            head_dim = layer.self_attn.head_dim
            # print(key_states.shape, num_key_value_heads)
            key_states = key_states.view(1, 1, num_key_value_heads, head_dim).transpose(1, 2)
            value_states = value_states.view(1, 1, num_key_value_heads, head_dim).transpose(1, 2)
            # print(student_cache[0][0].shape)
            current_position = student_cache[0][0].shape[-2]
            position_ids = torch.tensor([[current_position]], device=device)
            cos, sin = model.base_model.model.model.rotary_emb(value_states, position_ids)
            _, key_states = apply_rotary_pos_emb(key_states, key_states, cos, sin)
            # find_updated_cache(exit_hidden_state, layer, student_cache, current_position)
            # print(student_cache[layer_idx][0][:, :, -1:].shape, key_states.shape)
            # student_cache[layer_idx][0][:, :, -1] = student_cache[early_exit_layer][0][:, :, -1] # keys
            # student_cache[layer_idx][1][:, :, -1] = student_cache[early_exit_layer][1][:, :, -1] # values
            
            student_cache[layer_idx][0][:, :, -1:] = key_states # keys
            student_cache[layer_idx][1][:, :, -1:] = value_states # values
            
        # print(len(student_cache), student_cache[0][0].shape)
        
        teacher_outputs = model(
            next_token,
            past_key_values=teacher_cache,
            use_cache=True
        )
        teacher_cache = teacher_outputs.past_key_values
        teacher_logits = teacher_outputs.logits
        all_teacher_logits.append(teacher_logits)
    # Take the most likely next token (greedy decoding here)
    next_token = torch.argmax(student_logits[:, -1, :], dim=-1).unsqueeze(-1)
    # next_token = torch.argmax(teacher_logits[:, -1, :], dim=-1).unsqueeze(-1)
    if step > 0: 
        generated_tokens.append(next_token.item())
        chosen_exit_layers.append(early_exit_layer)
    else:
        generated_tokens.append(next_token.item())
        chosen_exit_layers.append(-1)
    # Append to generated sequence
    generated = torch.cat([generated, next_token], dim=-1)

print(tokenizer.decode(generated[0]))
all_student_logits = torch.concatenate(all_student_logits, axis = 0).transpose(0,1)
all_teacher_logits = torch.concatenate(all_teacher_logits, axis = 0).transpose(0,1)
# all_student_logits.shape
tokens = [tokenizer.decode([token]) for token in generated_tokens]
layers = [27 if item == 27 or item == -1 else item for item in chosen_exit_layers]
early_exit_layers = early_exit_layer_idxs.tolist()  # Convert tensor to list if needed
# Display the visualization
display(visualize_tokens_by_exit_layer(tokens, layers, early_exit_layers, 
                                     title="Committed Early Exit Token Generation"))

<｜begin▁of▁sentence｜><｜Assistant｜> You are a helpful programming tutor.
<｜User｜> Explain the concept of recursion in programming.
<｜Assistant｜> 
Okay, so I need to explain � "__新型冠状onium__" the concept of recursion in programming.


In [20]:
def propagate_exit_hidden_to_cache(model, exit_hidden_state, early_exit_layer, cache, current_position):
    """
    Propagate the same hidden state to all future layers' KV caches.
    
    exit_hidden_state: [batch, seq_len, hidden_dim] - the hidden state at early exit
    cache: the current KV cache
    current_position: the position in the sequence
    """
    with torch.no_grad():
        device = exit_hidden_state.device
        batch_size, seq_len, hidden_dim = exit_hidden_state.shape
        
        # For each layer after early exit
        for layer_idx in range(early_exit_layer + 1, len(model.base_model.model.model.layers)):
            layer = model.base_model.model.model.layers[layer_idx]
            
            # Apply layer norm to the EXIT hidden state
            normed_hidden = layer.input_layernorm(exit_hidden_state)
            
            # Project to K and V using this layer's projections
            key_states = layer.self_attn.k_proj(normed_hidden)
            value_states = layer.self_attn.v_proj(normed_hidden)
            
            # Reshape for multi-head attention
            num_key_value_heads = layer.self_attn.config.num_key_value_heads
            head_dim = layer.self_attn.head_dim
            
            key_states = key_states.view(batch_size, seq_len, num_key_value_heads, head_dim).transpose(1, 2)
            value_states = value_states.view(batch_size, seq_len, num_key_value_heads, head_dim).transpose(1, 2)
            
            # Apply RoPE to keys (Qwen2 needs this)
            position_ids = torch.tensor([[current_position]], device=device)

            key_states, key_states = apply_rotary_pos_emb(key_states, key_states, cos, sin)
            

            # Now update the cache directly
            # The cache is a list of tuples (key_cache, value_cache) for each layer
            old_key_cache, old_value_cache = cache[layer_idx]
            
            # Concatenate with existing cache
            # Note: we're only updating the last position, so we can either:
            # Option 1: Replace the last position (if it was already computed)
            # Option 2: Append if this is a new position
            
 
                old_key_cache[:, :, -1:, :] = key_states
                old_value_cache[:, :, -1:, :] = value_states
            else:
                # Multiple tokens - replace the last seq_len positions
                old_key_cache[:, :, -seq_len:, :] = key_states
                old_value_cache[:, :, -seq_len:, :] = value_states

            print(cache[layer_idx])
            # Update cache in place
            cache[layer_idx][0] = old_key_cache
            cache[layer_idx] = (old_key_cache, old_value_cache)
    
    return cache

# Your generation loop with the fix:
def set_early_exit_layer(step):
    if step == 2: return 25
    return 27

set_transformer_early_exit_mode(model, 'off')
generated_tokens = []
max_new_tokens = 10
generated = inputs["input_ids"]
chosen_exit_layers = []
all_student_logits = []
all_teacher_logits = []

for step in range(max_new_tokens):
    if step == 0:
        student_outputs = model(generated, use_cache=True, output_hidden_states=True)
        student_logits = student_outputs.logits
        student_cache = student_outputs.past_key_values
        teacher_outputs = model(generated, use_cache=True)
        teacher_cache = teacher_outputs.past_key_values
        teacher_logits = teacher_outputs.logits
    else:
        # Pass only the new token and the cache
        student_outputs = model(
            next_token,
            past_key_values=student_cache,
            use_cache=True,
            output_hidden_states=True
        )
        
        early_exit_layer = set_early_exit_layer(step)
        hidden_states = torch.stack(student_outputs.hidden_states)[1:]  # Skip embeddings
        
        # Get the hidden state at early exit layer for the last token
        exit_hidden_state = hidden_states[early_exit_layer][:, -1:, :]  # [batch, 1, hidden_dim]
        
        # Get student logits from early exit
        student_logits = model.lm_head(exit_hidden_state)
        all_student_logits.append(student_logits)
        
        # First, update the cache normally
        student_cache = student_outputs.past_key_values
        
        # Then propagate the exit hidden state to future layers' caches
        current_position = student_cache[0][0].shape[2] - 1
        student_cache = propagate_exit_hidden_to_cache(
            model,
            exit_hidden_state,
            early_exit_layer,
            student_cache,
            current_position
        )
        
        # Teacher forward pass (normal)
        teacher_outputs = model(
            next_token,
            past_key_values=teacher_cache,
            use_cache=True
        )
        teacher_cache = teacher_outputs.past_key_values
        teacher_logits = teacher_outputs.logits
        all_teacher_logits.append(teacher_logits)
    
    # Sample next token
    next_token = torch.argmax(teacher_logits[:, -1, :], dim=-1).unsqueeze(-1)
    
    if step > 0:
        generated_tokens.append(next_token.item())
        chosen_exit_layers.append(early_exit_layer)
    else:
        generated_tokens.append(next_token.item())
        chosen_exit_layers.append(-1)
    
    # Append to generated sequence
    generated = torch.cat([generated, next_token], dim=-1)

# Visualization code...

(tensor([[[[-2.1860e-02, -7.2696e-02, -1.1570e-01,  ...,  2.1059e+00,
            4.7061e-01, -7.4747e-01],
          [-1.0939e-02, -7.7366e-02, -4.7914e-02,  ...,  2.1022e+00,
            4.8923e-01, -7.4620e-01],
          [-4.1520e-01, -8.6979e-02, -3.5009e-01,  ...,  4.1750e+00,
           -2.6645e+00, -1.5610e+00],
          ...,
          [ 5.0791e-01, -1.4946e-01, -4.3302e-02,  ...,  1.7778e+00,
           -1.9287e+00, -1.1862e+00],
          [-1.3289e-01, -1.6452e-01, -3.5462e-02,  ..., -1.1493e+00,
           -1.1632e+00, -1.9846e+00],
          [ 6.9239e-01,  2.4655e-01,  7.8054e-01,  ...,  1.5079e+00,
           -8.3180e-01, -3.5837e+00]],

         [[ 3.6603e-03, -1.3313e-03,  3.6746e-02,  ..., -1.3509e+00,
           -1.1811e+00,  3.5928e+00],
          [ 7.6382e-04, -9.9849e-03,  4.2674e-02,  ..., -1.3672e+00,
           -1.1651e+00,  3.5925e+00],
          [ 3.8066e-01, -9.2743e-02, -4.8343e-01,  ...,  2.7600e+00,
           -2.9237e+00, -1.8019e+00],
          ...,
    

TypeError: 'tuple' object does not support item assignment

In [8]:
chosen_exit_layers_tensor = torch.tensor(chosen_exit_layers[1:], device=device).unsqueeze(0).float()
chosen_exit_layers_tensor = torch.where(
                    chosen_exit_layers_tensor == 27,
                    torch.full_like(chosen_exit_layers_tensor, float('inf')),
                    chosen_exit_layers_tensor
                )
repeated_sft_teacher_generated_tokens = generated[:, :-1]

set_transformer_early_exit_mode(model, 'sft_student')
sft_student_output_scores, collected_exit_logits = model(repeated_sft_teacher_generated_tokens,\
                                                             prescribed_exit_layer_idxs=chosen_exit_layers_tensor)

sft_student_output = sft_student_output_scores.logits.squeeze()[20:]
kv_cache_output = all_student_logits.squeeze()

student_probs = F.softmax(sft_student_output, dim=-1)
teacher_cache_probs = F.softmax(all_teacher_logits.squeeze(), dim=-1)
student_cache_probs = F.softmax(kv_cache_output, dim=-1)

# model_outputs = model(repeated_sft_teacher_generated_tokens)
# model_probs = F.softmax(model_outputs.logits[0, 20:], dim = -1)

# set_transformer_early_exit_mode(frozen_model, 'off')
# off_outputs = frozen_model(repeated_sft_teacher_generated_tokens)
# off_probs = F.softmax(off_outputs.logits[0, 20:], dim = -1)

pd.options.display.float_format = "{:.2f}".format
rows = []

def get_prob_token(probs):
    top_id = torch.argmax(probs).item()
    top_prob = probs[top_id].item()
    top_token = tokenizer.decode([top_id])
    return top_prob, top_token

for idx in range(len(student_probs)):
    # Student
    student_top_prob, student_top_token = get_prob_token(student_probs[idx])
    # teacher_top_prob, teacher_top_token = get_prob_token(teacher_probs[idx])
    
    student_cache_top_prob, student_cache_top_token = get_prob_token(student_cache_probs[idx])
    teacher_cache_top_prob, teacher_cache_top_token = get_prob_token(teacher_cache_probs[idx])

    
    # model_top_prob, model_top_token = get_prob_token(model_probs[idx])
    
    # off_top_prob, off_top_token = get_prob_token(off_probs[idx])

    rows.append({
        # "Position": idx,
        "Student Token": student_top_token,
        "Student Prob": student_top_prob,
        # "Teacher Token": teacher_top_token,
        # "Teacher Prob": teacher_top_prob,
        "Student Cache Token": student_cache_top_token,
        "Student Cache Prob": student_cache_top_prob,
        "Teacher Cache Token": teacher_cache_top_token,
        "Teacher Cache Prob": teacher_cache_top_prob,
        # "Model Token": model_top_token,
        # "Model Prob": model_top_prob,
        # "Off Token": off_top_token,
        # "Off Prob": off_top_prob
    })

df = pd.DataFrame(rows)
display(df)

,Student Token,Student Prob,Student Cache Token,Student Cache Prob,Teacher Cache Token,Teacher Cache Prob
0,",",1.00,",",1.00,",",1.00
1,so,0.99,so,1.00,so,0.78
2,Helvetica,0.01,I,0.92,I,0.93
3,一定程度,0.01,need,0.64,need,0.65
4,transparent,0.03,to,1.00,to,1.00
5,lay,0.01,explain,0.74,explain,0.75
6,enses,0.01,recursion,0.50,recursion,0.50
7,Helvetica,0.00,in,0.98,in,0.98
8,incoming,0.06,programming,1.00,programming,1.00
